In [ ]:
import sys
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from fpbinary import FpBinary, FpBinaryComplex, RoundingEnum, OverflowEnum
%matplotlib widget

In [ ]:
sys.path.append("../../utils")
from fi_fir import fi_fir

In [ ]:
def plot_fft(x, Fs, Fs_units='', n=None):
    if n is None:
        n = len(x)
    else:
        n = int(n)

    if Fs_units == '':
        Fs_scale = 1
    elif Fs_units == 'k':
        Fs_scale = 1/1e3
    elif Fs_units == 'M':
        Fs_scale = 1/1e6
    else:
        raise(ValueError("Unsupported Fs Unit"))

    X = np.fft.fft(x[0:n])
    f = np.fft.fftfreq(X.size, d=1/Fs)

    fig = plt.figure(clear=True)
    plt.plot(np.fft.fftshift(f*Fs_scale), 20*np.log10(np.abs(np.fft.fftshift(X))))
    plt.xlabel(f'Frequency ({Fs_units}Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.grid()

In [ ]:
# Load input signal
Fs = 1.92e6 # Hz
filename = "/home/hicksze1/antsdr-pynq/boards/e200/fm_radio/sim/iq_files/fm_1920kHz.fc32"
# filename = "/home/hicksze1/antsdr-pynq/boards/e200/fm_radio/sim/iq_files/tb_chirp.ic16"
IQ_interleaved = np.fromfile(filename, dtype=np.float32)
IQ = IQ_interleaved[::2] + 1j*IQ_interleaved[1::2]
IQ = IQ[:4*1920000]
del IQ_interleaved

In [ ]:
# Design decimation filters

# Stage 1
M1 = 5 # decimation factor
Fs_1 = Fs / M1 # output sample rate
N1 = 21 # filter order
delta_w = 1 / 100 # transition width
wp = (1/M1) - delta_w # passband freq
ws = (1/M1) + delta_w # stopband freq
h_decim_1 = sig.remez(N1, [0, wp, ws, 0.5], [1, 0], fs=1)
H_decim_1 = sig.dlti(h_decim_1, [1], dt = 1/Fs)

# Stage 2
M2 = 4 # decimation factor
Fs_2 = Fs_1 / M2 # output sample rate
N2 = 41 # filter order
delta_w = 1 / 100 # transition width
wp = (1/M2) - delta_w # passband freq
ws = (1/M2) + delta_w # stopband freq
h_decim_2 = sig.remez(N2, [0, wp, ws, 0.5], [1, 0], fs=1)
H_decim_2 = sig.dlti(h_decim_2, [1], dt = 1/Fs_1)

# Stage 3
M3 = 2 # decimation factor
Fs_3 = Fs_2 / M3 # output sample rate
N3 = 41 # filter order
h_decim_3 = sig.remez(N3, [0, 16e3, 20e3, Fs_2/2], [1, 0], fs=Fs_2)
H_decim_3 = sig.dlti(h_decim_3, [1], dt = 1/Fs_2)

In [ ]:
IQ_decim, _ = fi_fir(IQ, h_decim_1, M=M1)

In [ ]:
plot_fft(IQ_decim, Fs_1)

In [ ]:
# Instantaneous frequency estimation
h_diff = [1, 0, -1]
IQ_diff, _ = fi_fir(IQ_decim, h_diff)

In [ ]:
I_decim = np.array([val.real for val in IQ_decim])
Q_decim = np.array([val.imag for val in IQ_decim])
I_diff = np.array([val.real for val in IQ_diff])
Q_diff = np.array([val.imag for val in IQ_diff])

In [ ]:
prod1 = I_decim * Q_diff[:-2]
prod2 = Q_decim * I_diff[:-2]

In [ ]:
freq = (prod1 - prod2)
freq = np.array([val.resize((1, 31), round_mode=RoundingEnum.near_even) for val in freq])

In [ ]:
freq_decim, _ = fi_fir(freq, h_decim_2, x_q=(1, 31), y_q=(1, 31), M=M2, real=True)
audio, _ = fi_fir(freq_decim, h_decim_3, x_q=(1, 31), y_q=(1, 31), M=M3, real=True)

In [ ]:
tau = 75e-6

wc = 1 / tau
wca = 2 * Fs_3 * np.tan(wc / (2*Fs_3))
k = -wca / (2*Fs_3)
z1 = -1
p1 = (1 + k) / (1 - k)
b0 = -k / (1 - k)
b = [b0, b0*-z1]
a = [1, -p1]

In [ ]:
audio_float = np.array([float(val) for val in audio])
deemph = sig.lfilter(b, a, audio_float)
deemph_dc_blocked = deemph - np.mean(deemph)

In [ ]:
from IPython.display import Audio

Audio(deemph_dc_blocked, rate=Fs_3)